In [3]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [4]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

IMAGE_SIZE = [480,270]

train_path = '../input/updated-dataset/Updated_Dataset/Train'
valid_path = '../input/updated-dataset/Updated_Dataset/Test'

#training_data, testing_data = train_test_split(df, test_size=0.2, random_state=25)
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False
  

  
  # useful for getting number of classes
folders = glob('../input/updated-dataset/Updated_Dataset/Train/*')
  

# our layers - you can add more if you want
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders)-1, activation='sigmoid')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()


model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('../input/updated-dataset/Updated_Dataset/Train',
                                                 target_size = (480,270),
                                                 batch_size = 3,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('../input/updated-dataset/Updated_Dataset/Test',
                                            target_size = (480,270),
                                            batch_size = 3,
                                            class_mode = 'binary')

'''r=model.fit_generator(training_set,
                         samples_per_epoch = 8000,
                         nb_epoch = 5,
                         validation_data = test_set,
                         nb_val_samples = 2000)'''

# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=1
)
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

import tensorflow as tf

from keras.models import load_model

model.save('facefeatures_new_model.h5')

In [23]:
import cv2
from keras.preprocessing.image import img_to_array
image = cv2.imread('../input/updated-dataset/Updated_Dataset/Test/No/Frame313.jpg')
image = image.reshape((1, image.shape[1], image.shape[0], image.shape[2]))
yhat = model.predict(image)
if yhat[0][0] > 0.5:
    label = 1
else:
    label = 0
label